In [1]:
import findspark
findspark.init("c:\spark")

In [2]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext

In [3]:
conf = SparkConf() \
.setMaster("local[4]") \
.setAppName("Test") \
.setExecutorEnv("spark.executor.memory" , "3g") \
.setExecutorEnv("spark.driver.memory" , "2g")

sc = SparkContext(conf=conf)

dataRDD = sc.textFile("C:\\Users\\MERT\\Untitled Folder\\Data\\OnlineRetail.csv")

In [7]:
dataRDD.top(3) 

['InvoiceNo;StockCode;Description;Quantity;InvoiceDate;UnitPrice;CustomerID;Country',
 'C581569;84978;HANGING HEART JAR T-LIGHT HOLDER;-1;9.12.2011 11:58;1,25;17315;United Kingdom',
 'C581569;20979;36 PENCILS TUBE RED RETROSPOT;-5;9.12.2011 11:58;1,25;17315;United Kingdom']

## Sütünları temizlemenin farklı yolu ınvoıce no olmayan satırı al dıyoruz o tarzında misalisi

In [8]:
dataRDD2 =dataRDD.filter(lambda x:"InvoiceNo" not in x)
dataRDD2.take(3)

['536365;85123A;WHITE HANGING HEART T-LIGHT HOLDER;6;1.12.2010 08:26;2,55;17850;United Kingdom',
 '536365;71053;WHITE METAL LANTERN;6;1.12.2010 08:26;3,39;17850;United Kingdom',
 '536365;84406B;CREAM CUPID HEARTS COAT HANGER;8;1.12.2010 08:26;2,75;17850;United Kingdom']

In [9]:
##miktar la unit price i çarpıcaz toplıcaz onları kaç tane iptal oldu anlıcaz

In [15]:
def cancalled_price(line):
    if(line.split(";")[0].startswith("C")):
        is_cancalled = True
    else:
        is_cancalled = False
        
    quantity = float(line.split(";")[3])
    price = float(line.split(";")[5].replace(",","."))
    total = quantity * price
    
    return (is_cancalled , total)

In [16]:
cancalled_dataRDD3 = dataRDD2.map(cancalled_price)

In [18]:
cancalled_dataRDD3.take(10)

[(False, 15.299999999999999),
 (False, 20.34),
 (False, 22.0),
 (False, 20.34),
 (False, 20.34),
 (False, 15.3),
 (False, 25.5),
 (False, 11.100000000000001),
 (False, 11.100000000000001),
 (False, 54.08)]

In [21]:
cancalled_dataRDD3_reduce =cancalled_dataRDD3.reduceByKey(lambda x,y : x+y)


In [26]:
cancalled_dataRDD3_reduce.top(3)

[(True, -896812.4900000116), (False, 10644560.424000263)]

In [29]:
c2 = cancalled_dataRDD3_reduce.filter(lambda x : x[0] == True)

In [30]:
c2.top(3)

[(True, -896812.4900000116)]

In [33]:
c3 = c2.map(lambda x : x[1])

In [34]:
c3.top(3)

[-896812.4900000116]